<a href="https://colab.research.google.com/github/Shreyans06/LLMs-understanding/blob/main/tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get the text file


In [9]:
import urllib
url =  ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "verdict.txt"
urllib.request.urlretrieve(url , file_path)

('verdict.txt', <http.client.HTTPMessage at 0x7e4a2d142fc0>)

# Read the text file

In [10]:
with open("verdict.txt" , "r" , encoding="utf-8") as f:
  raw_text = f.read()

print("Total characters:" , len(raw_text))
print(raw_text[:50])

Total characters: 20479
I HAD always thought Jack Gisburn rather a cheap g


# Pre-process the text file

In [11]:
import re

pre_process = re.split(r'([,.:;?!"()_\']|--|\s)', raw_text)
pre_processed = [item.strip() for item in pre_process if item.strip()]
print(len(pre_processed))

4690


# Sample pre-processed data

In [12]:
print(pre_processed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


# Create a vocabulary

In [13]:
words = sorted(set(pre_processed))
vocab_size = len(words)
print(vocab_size)

1130


In [14]:
vocab = {token : integer for integer , token in enumerate(words)}
for i , item in enumerate(vocab.items()):
  print(item)
  if i >= 30:
    break


('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)


# Text Tokenizer

In [15]:
class TokenizerV1:
  def __init__(self , vocab):
    self.str_to_int = vocab
    self.int_to_str = {integer : string for string , integer in vocab.items() }

  def encode(self, text):
    preprocess = re.split(r'([,.()?_"!\']|--|\s)' , text)
    preprocessed = [ item.strip() for item in preprocess if item.strip()]
    ids = [ self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self , ids):
    text = " ".join([self.int_to_str[i] for i in ids])

    text = re.sub(r'\s+([,.()?!\'])' , r'\1' , text)
    return text

In [16]:
tokenizer = TokenizerV1(vocab)
text = """"It's the last he painted, you know,"
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [17]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know, " Mrs. Gisburn said with pardonable pride.
